<a href="https://colab.research.google.com/github/jioh-kim/ML-Audio-to-Pitch/blob/main/Hyperparametertuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install requirements

In [ ]:
!pip install tensorflow tensorflow-datasets librosa

Import libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import librosa
import numpy as np

Only grab the dataset where instument == keyboard

In [ ]:
# Constants
KEYBOARD_FAMILY_LABEL = 4 # According to NSynth dataset family label
SAMPLE_RATE = 16000
TRIM_LENGTH = 3 * SAMPLE_RATE  # Trim to the first 3 seconds

def process_data(example):
    audio = example['audio']
    instrument_family = example['instrument']['family']
    pitch = example['pitch']

    is_keyboard = tf.equal(instrument_family, KEYBOARD_FAMILY_LABEL)

    def process_keyboard_sample(audio, pitch):
        audio = audio[:TRIM_LENGTH]
        if pitch < 21:
            # Instead of returning None, return a marker (e.g., a zero-length tensor)
            return tf.zeros((0,)), tf.constant(-1, dtype=tf.int64)
        else:
            pitch = pitch - 21
            return audio, pitch

    return tf.cond(is_keyboard, lambda: process_keyboard_sample(audio, pitch), lambda: (audio, pitch))

def filter_keyboard_samples(example):
    return tf.equal(example['instrument']['family'], KEYBOARD_FAMILY_LABEL)

def filter_invalid_samples(audio, pitch):
    # Check if the sample is valid (not marked for removal)
    return tf.size(audio) > 0 and tf.not_equal(pitch, -1)

def get_data_loader(data_split, batch_size=64, num_batches=None):
    ds = tfds.load('nsynth', split=data_split, as_supervised=False)

    ds = ds.filter(filter_keyboard_samples)
    ds = ds.map(process_data, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.filter(filter_invalid_samples)
    ds = ds.batch(batch_size)
    if num_batches:
        ds = ds.take(num_batches)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)

    return ds

In [ ]:
# Input to model
batch_size = 8
train_loader = get_data_loader('train', batch_size, num_batches=40)
val_loader = get_data_loader('valid', batch_size, num_batches=6)
test_loader = get_data_loader('test', batch_size, num_batches=6)
classes = list(range(88))

for audio, pitch in train_loader.take(5):
    print(pitch)

Dl Completed...:   0%|          | 0/1069 [00:00<?, ? file/s]

Dataset nsynth downloaded and prepared to /root/tensorflow_datasets/nsynth/full/2.3.3. Subsequent calls will reuse this data.
tf.Tensor([85 10 60 33 48 84 21 54], shape=(8,), dtype=int64)
tf.Tensor([13 33  0  6 49 52 56 17], shape=(8,), dtype=int64)
tf.Tensor([ 4 74  6 40 63 20 44 41], shape=(8,), dtype=int64)
tf.Tensor([ 7 22 67 64  8 78 16 48], shape=(8,), dtype=int64)
tf.Tensor([14 29 87 32 70 84 65 66], shape=(8,), dtype=int64)


In [ ]:
# printing a sample audio and pitch value
for audio, pitch in train_loader.take(1):
    first_sample_audio = audio[0]
    first_sample_pitch = pitch[0]

    # Calculate the maximum and minimum values
    max_value = tf.reduce_max(first_sample_audio)
    min_value = tf.reduce_min(first_sample_audio)

    # Calculate the range (max - min)
    range_value = max_value - min_value

    print(f"First sample audio max value: {max_value.numpy()}")
    print(f"First sample audio min value: {min_value.numpy()}")
    print(f"First sample audio range: {range_value.numpy()}")
    print(f"First sample pitch: {first_sample_pitch.numpy()}")

First sample audio max value: 0.46275418996810913
First sample audio min value: -0.46886491775512695
First sample audio range: 0.9316191077232361
First sample pitch: 85


In [ ]:
for audio, pitch in train_loader:
  print(audio.shape)
  print(pitch.shape)


(64, 48000)
(64,)
(64, 48000)
(64,)
(64, 48000)
(64,)
(64, 48000)
(64,)
(64, 48000)
(64,)
(64, 48000)
(64,)
(64, 48000)
(64,)
(64, 48000)
(64,)
(64, 48000)
(64,)
(64, 48000)
(64,)


In [ ]:
# get number of batches in each loader
def get_dataset_length(data_loader):
    length = 0
    for _ in data_loader:
        length += 1
    return length

# Use this function to get the length of your data loaders
test_loader_length = get_dataset_length(test_loader)
val_loader_length = get_dataset_length(val_loader)
train_loader_length = get_dataset_length(train_loader)

print(f"Train loader length: {train_loader_length}")
print(f"Validation loader length: {val_loader_length}")
print(f"Test loader length: {test_loader_length}")

Train loader length: 40
Validation loader length: 6
Test loader length: 6


In [ ]:
# get number of samples in each loader
def get_dataset_sample_count(data_loader):
    total_samples = 0
    for audio, pitch in data_loader:
        # Count the number of samples in each batch
        batch_samples = tf.shape(audio)[0]  # assuming audio is a 2D tensor [batch_size, features]
        total_samples += batch_samples
    return total_samples

# Use this function to get the number of samples in your data loaders
test_samples_count = get_dataset_sample_count(test_loader)
val_samples_count = get_dataset_sample_count(val_loader)
train_samples_count = get_dataset_sample_count(train_loader)

print(f"Train loader samples: {train_samples_count}")
print(f"Validation loader samples: {val_samples_count}")
print(f"Test loader samples: {test_samples_count}")

Train loader samples: 320
Validation loader samples: 48
Test loader samples: 48


In [ ]:
################################################################################################################################################################
####################################################################### Model and Training #####################################################################
################################################################################################################################################################

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

torch.manual_seed(10)

In [ ]:
for audio, pitch in train_loader.take(1):
  print(f"Audio shape: {audio.shape}")

Audio shape: (64, 48000)


In [ ]:
# Model architecture

'''
class PitchDetectionModel(nn.Module):
  def __init__(self):
    super(PitchDetectionModel, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=1,     out_channels=1024,  kernel_size=4, stride=4)
    self.conv2 = nn.Conv1d(in_channels=1024,  out_channels=128,   kernel_size=4, stride=4)
    self.conv3 = nn.Conv1d(in_channels=128,   out_channels=128,   kernel_size=4, stride=4)
    self.conv4 = nn.Conv1d(in_channels=128,   out_channels=256,   kernel_size=2, stride=2)
    self.pool  = nn.MaxPool1d(2, 2)
    self.fc1   = nn.Linear(256*23, 88)

  def forward(self, x):                   # Input length 48,000,  channels 1
    x = self.pool(F.relu(self.conv1(x)))  #       length 6,000,   channels 1024
    x = self.pool(F.relu(self.conv2(x)))  #       length 750,     channels 128
    x = self.pool(F.relu(self.conv3(x)))  #       length 93,      channels 128
    x = self.pool(F.relu(self.conv4(x)))  #       length 23,      channels 256
    x = x.view(-1, 256*23)                # Flatten the tensor
    x = self.fc1(x)                       # Output length 88
    # x = F.softmax(self.fc1(x))
    return x                              # Assuming CELoss
'''

In [ ]:
from keras import layers, models

In [ ]:
'''
class PitchDetectionModel(tf.keras.Model):
  def __init__(self):
    super(PitchDetectionModel, self).__init__()
    self.conv1 = tf.keras.layers.Conv1D(in_channels=1,     out_channels=1024,  kernel_size=4, stride=4)
    self.conv2 = tf.keras.layers.Conv1D(in_channels=1024,  out_channels=128,   kernel_size=4, stride=4)
    self.conv3 = tf.keras.layers.Conv1D(in_channels=128,   out_channels=128,   kernel_size=4, stride=4)
    self.conv4 = tf.keras.layers.Conv1D(in_channels=128,   out_channels=256,   kernel_size=2, stride=2)
    self.pool  = tf.keras.layers.MaxPool1D(2, 2)
    self.fc1   = tf.keras.layers.Dense(88)

  def call(self, x):                   # Input length 48,000,  channels 1
    x = self.pool(tf.nn.relu(self.conv1(x)))  #       length 6,000,   channels 1024
    x = self.pool(tf.nn.relu(self.conv2(x)))  #       length 750,     channels 128
    x = self.pool(tf.nn.relu(self.conv3(x)))  #       length 93,      channels 128
    x = self.pool(tf.nn.relu(self.conv4(x)))  #       length 23,      channels 256
    x = tf.reshape(x, (-1, 256*23))                # Flatten the tensor
    x = self.fc1(x)                       # Output length 88
    # x = tf.nn.softmax(self.fc1(x))
    return x                              # Assuming CELoss

model = PitchDetectionModel()
'''

In [ ]:

class PitchDetectionModel(tf.keras.Model):
    def __init__(self):
        super(PitchDetectionModel, self).__init__()
        self.reshape = layers.Reshape((48000, 1))
        self.conv1 = layers.Conv1D(1024, kernel_size=4, strides=4, activation='relu')
        self.conv2 = layers.Conv1D(128, kernel_size=4, strides=4, activation='relu')
        self.conv3 = layers.Conv1D(128, kernel_size=4, strides=4, activation='relu')
        self.conv4 = layers.Conv1D(256, kernel_size=2, strides=2, activation='relu')
        self.pool = layers.MaxPooling1D(2)
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(88)

    def call(self, x):
        x = self.reshape(x)
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        x = self.pool(self.conv3(x))
        x = self.pool(self.conv4(x))
        x = self.flatten(x)
        return self.fc1(x)


In [ ]:
'''
class PitchDetectionModel(tf.keras.Model):
  def __init__(self):
    super(PitchDetectionModel, self).__init__()
    self.reshape = layers.Reshape((48000, 1))
    self.conv1 = layers.Conv1D(1024, kernel_size=4, strides=4, activation='relu')
    self.conv2 = layers.Conv1D(128, kernel_size=4, strides=4, activation='relu')
    self.conv3 = layers.Conv1D(128, kernel_size=4, strides=4, activation='relu')
    self.conv4 = layers.Conv1D(256, kernel_size=2, strides=2, activation='relu')
    self.pool = layers.MaxPooling1D(2)
    self.flatten = layers.Flatten()
    self.fc1 = layers.Dense(88)

  def call(self, x):                   # Input length 48,000,  channels 1
    x = self.pool(tf.nn.relu(self.conv1(x)))  #       length 6,000,   channels 1024
    x = self.pool(tf.nn.relu(self.conv2(x)))  #       length 750,     channels 128
    x = self.pool(tf.nn.relu(self.conv3(x)))  #       length 93,      channels 128
    x = self.pool(tf.nn.relu(self.conv4(x)))  #       length 23,      channels 256
    x = tf.reshape(x, (-1, 256*23))                # Flatten the tensor
    x = self.fc1(x)                       # Output length 88
    # x = tf.nn.softmax(self.fc1(x))
    return x                              # Assuming CELoss
'''

"\nclass PitchDetectionModel(tf.keras.Model):\n  def __init__(self):\n    super(PitchDetectionModel, self).__init__()\n    self.reshape = layers.Reshape((48000, 1))  \n    self.conv1 = layers.Conv1D(1024, kernel_size=4, strides=4, activation='relu')\n    self.conv2 = layers.Conv1D(128, kernel_size=4, strides=4, activation='relu')\n    self.conv3 = layers.Conv1D(128, kernel_size=4, strides=4, activation='relu')\n    self.conv4 = layers.Conv1D(256, kernel_size=2, strides=2, activation='relu')\n    self.pool = layers.MaxPooling1D(2)\n    self.flatten = layers.Flatten()\n    self.fc1 = layers.Dense(88)\n\n  def call(self, x):                   # Input length 48,000,  channels 1\n    x = self.pool(tf.nn.relu(self.conv1(x)))  #       length 6,000,   channels 1024\n    x = self.pool(tf.nn.relu(self.conv2(x)))  #       length 750,     channels 128\n    x = self.pool(tf.nn.relu(self.conv3(x)))  #       length 93,      channels 128\n    x = self.pool(tf.nn.relu(self.conv4(x)))  #       length 23

In [ ]:
# Create an instance of the model
model = PitchDetectionModel()

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train the model
# history = model.fit(train_loader, epochs=10, validation_data=val_loader)

print("Fit model on training data")
history = model.fit(
  x=train_loader,
  epochs=10,
  verbose=2,
  validation_data=val_loader
)
'''
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)
'''
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_loader)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')

Fit model on training data
Epoch 1/10
40/40 - 15s - loss: 4.4835 - sparse_categorical_accuracy: 0.0125 - val_loss: 4.4686 - val_sparse_categorical_accuracy: 0.0000e+00 - 15s/epoch - 386ms/step
Epoch 2/10
40/40 - 3s - loss: 4.3076 - sparse_categorical_accuracy: 0.0250 - val_loss: 4.4129 - val_sparse_categorical_accuracy: 0.0000e+00 - 3s/epoch - 83ms/step
Epoch 3/10
40/40 - 3s - loss: 4.1243 - sparse_categorical_accuracy: 0.0594 - val_loss: 4.6557 - val_sparse_categorical_accuracy: 0.0208 - 3s/epoch - 83ms/step
Epoch 4/10
40/40 - 3s - loss: 3.8858 - sparse_categorical_accuracy: 0.0750 - val_loss: 5.0740 - val_sparse_categorical_accuracy: 0.0625 - 3s/epoch - 83ms/step
Epoch 5/10
40/40 - 3s - loss: 3.4776 - sparse_categorical_accuracy: 0.1312 - val_loss: 5.5558 - val_sparse_categorical_accuracy: 0.0417 - 3s/epoch - 83ms/step
Epoch 6/10
40/40 - 3s - loss: 2.9314 - sparse_categorical_accuracy: 0.2406 - val_loss: 6.2727 - val_sparse_categorical_accuracy: 0.1042 - 3s/epoch - 84ms/step
Epoch 7/